In [1]:
import os
import contextlib
import glob
import random

import numpy as np
import pandas as pd

# Used for stratified train/val split.
try:
    from sklearn.model_selection import train_test_split
except ImportError as e:
    raise ImportError(
        "scikit-learn is required for the stratified train/val split. "
        "Install it with: pip install scikit-learn"
    ) from e

# Quieter TF logs + friendlier GPU memory behavior.
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
os.environ.setdefault("TF_FORCE_GPU_ALLOW_GROWTH", "true")

@contextlib.contextmanager
def silence_stderr(enabled: bool = True):
    """Temporarily silence native stderr (useful for noisy TF/CUDA init logs)."""
    if not enabled:
        yield
        return
    fd = os.dup(2)
    try:
        with open(os.devnull, "w") as devnull:
            os.dup2(devnull.fileno(), 2)
            yield
    finally:
        os.dup2(fd, 2)
        os.close(fd)

SILENCE_TF_CPP_LOGS = True

with silence_stderr(SILENCE_TF_CPP_LOGS):
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Flatten, Dense
    from tensorflow.keras.applications.vgg16 import preprocess_input

    # Initialize GPU early so later cells are quieter.
    _ = tf.config.list_physical_devices("GPU")

tf.get_logger().setLevel("ERROR")

from PIL import Image
import matplotlib.image as img
import matplotlib.pyplot as plt


# Setting up the path and loading csv files

In [2]:
DATASET_ROOT = "../input/human-action-recognition-har-dataset/Human Action Recognition"
TRAIN_DIR = os.path.join(DATASET_ROOT, "train")
TEST_DIR = os.path.join(DATASET_ROOT, "test")

train_csv = pd.read_csv(os.path.join(DATASET_ROOT, "Training_set.csv"))
test_csv = pd.read_csv(os.path.join(DATASET_ROOT, "Testing_set.csv"))


In [3]:
train_fol = glob.glob(os.path.join(TRAIN_DIR, "*"))
test_fol = glob.glob(os.path.join(TEST_DIR, "*"))


In [ ]:
train_csv

In [ ]:
train_csv.label.value_counts()

In [ ]:
import plotly.express as px
l = train_csv.label.value_counts()
fig = px.pie(train_csv, values=l.values, names=l.index, title='Distribution of Human Activity')
fig.show()

In [7]:
train_filenames = train_csv["filename"]
train_labels_series = train_csv["label"]


In [ ]:
train_filenames


In [ ]:
train_labels_series


# Creating a function to random take a image and display it with its label

In [10]:
def randomImage():
    """Display a random training image (if it exists) and title it with its label."""
    num = random.randint(1, 10000)
    filename = f"Image_{num}.jpg"
    path = os.path.join(TRAIN_DIR, filename)

    if os.path.exists(path):
        image = img.imread(path)
        plt.imshow(image)
        label = train_csv.loc[train_csv["filename"] == filename, "label"].item()
        plt.title(label)
    else:
        print("File Path not found \nSkipping the file!!")


In [ ]:
randomImage()

In [ ]:
randomImage()

In [ ]:
randomImage()

# Processing data

In [14]:
train_image_paths = [os.path.join(TRAIN_DIR, fn) for fn in train_csv["filename"].tolist()]
train_labels = train_csv["label"].tolist()


In [ ]:
label_ids, class_names = pd.factorize(pd.Series(train_labels))
index_to_label = {int(i): name for i, name in enumerate(class_names)}
num_classes = len(class_names)

SEED = 42
VAL_SPLIT = 0.2

# 80/20 stratified split so validation metrics reflect the real class distribution.
all_paths = np.asarray(train_image_paths)
all_label_ids = np.asarray(label_ids)

train_paths, val_paths, train_label_ids, val_label_ids = train_test_split(
    all_paths,
    all_label_ids,
    test_size=VAL_SPLIT,
    random_state=SEED,
    stratify=all_label_ids,
)

# Keep labels consistently typed (we also cast again in the tf.data pipeline).
train_label_ids = train_label_ids.astype(np.int32)
val_label_ids = val_label_ids.astype(np.int32)

IMG_SIZE = (160, 160)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Stream images from disk (lower RAM) and prefetch for throughput.
def load_and_preprocess(path, label):
    img_bytes = tf.io.read_file(path)
    img = tf.io.decode_jpeg(img_bytes, channels=3)
    img = tf.image.resize(img, IMG_SIZE, antialias=True)
    img = preprocess_input(tf.cast(img, tf.float32))
    return img, tf.cast(label, tf.int32)

train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_label_ids))
train_ds = train_ds.shuffle(
    buffer_size=min(int(len(train_paths)), 2048),
    seed=SEED,
    reshuffle_each_iteration=True,
)
train_ds = train_ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_label_ids))
val_ds = val_ds.map(load_and_preprocess, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)


In [ ]:
vgg_model = Sequential()

pretrained_model = tf.keras.applications.VGG16(
    include_top=False,
    input_shape=(*IMG_SIZE, 3),
    pooling="avg",
    weights="imagenet",
)

for layer in pretrained_model.layers:
    layer.trainable = False

vgg_model.add(pretrained_model)
vgg_model.add(Flatten())
vgg_model.add(Dense(512, activation="relu"))
vgg_model.add(Dense(num_classes, activation="softmax"))


In [17]:
vgg_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vgg_model.summary()

In [ ]:
with silence_stderr(SILENCE_TF_CPP_LOGS):
    history = vgg_model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=60,
    )


In [20]:
vgg_model.save_weights("model.weights.h5")

In [ ]:
losss = history.history['loss']
plt.plot(losss)

In [ ]:
accu = history.history['accuracy']
plt.plot(accu)

# Custom Testing

In [23]:
# Read one RGB image and resize.

def read_image(fn):
    image = Image.open(fn).convert("RGB")
    return np.asarray(image.resize((160, 160)))

In [24]:
# Predict and show the image.

def test_predict(test_image):
    x = np.asarray([read_image(test_image)], dtype=np.float32)
    x = preprocess_input(x)
    result = vgg_model.predict(x)

    prediction = int(np.argmax(result, axis=1)[0])
    prob = float(np.max(result))
    label = index_to_label.get(prediction, prediction) if "index_to_label" in globals() else prediction

    print(f"probability: {prob * 100:.2f}%\nPredicted class: {label}")

    image = img.imread(test_image)
    plt.imshow(image)
    plt.title(label)

In [ ]:
test_predict('../input/human-action-recognition-har-dataset/Human Action Recognition/test/Image_1010.jpg')

In [ ]:
test_predict('../input/human-action-recognition-har-dataset/Human Action Recognition/test/Image_101.jpg')

In [ ]:
test_predict('../input/human-action-recognition-har-dataset/Human Action Recognition/test/Image_1050.jpg')